## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-06-22-43-32 +0000,nyt,At Least 17 Migrants Die in Greece’s Worst Shi...,https://www.nytimes.com/2025/12/06/world/europ...
1,2025-12-06-22-35-35 +0000,startribune,Four things to watch with the Twins at basebal...,https://www.startribune.com/four-things-to-wat...
2,2025-12-06-22-23-51 +0000,nypost,Private LA school ‘kissing club’ was hive of b...,https://nypost.com/2025/12/06/us-news/private-...
3,2025-12-06-22-20-35 +0000,startribune,Republican state senator admonishes Trump for ...,https://www.startribune.com/republican-state-s...
4,2025-12-06-21-41-29 +0000,nypost,Christmas returns to Bethlehem with tree-light...,https://nypost.com/2025/12/06/world-news/chris...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2345/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
29,trump,26
181,new,10
164,up,8
89,fire,8
73,you,7


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
125,2025-12-06-05-14-15 +0000,nypost,White House slams Newsom after gov claims Trum...,https://nypost.com/2025/12/06/us-news/white-ho...,60
10,2025-12-06-20-49-19 +0000,nypost,Qatari leader warns Gaza cease-fire at ‘critic...,https://nypost.com/2025/12/06/world-news/qatar...,59
31,2025-12-06-19-04-27 +0000,wapo,Trump urges a new vaccine schedule. Here’s wha...,https://www.washingtonpost.com/world/2025/12/0...,53
95,2025-12-06-11-45-00 +0000,nypost,"Zelensky, Trump admin have ‘constructive’ phon...",https://nypost.com/2025/12/06/world-news/russi...,51
153,2025-12-06-00-49-30 +0000,nypost,This is what Trump told Carney and Sheinbaum a...,https://nypost.com/2025/12/05/us-news/lip-read...,48


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
125,60,2025-12-06-05-14-15 +0000,nypost,White House slams Newsom after gov claims Trum...,https://nypost.com/2025/12/06/us-news/white-ho...
127,43,2025-12-06-04-27-00 +0000,wsj,A key vaccine panel at the Centers for Disease...,https://www.wsj.com/politics/policy/cdc-panel-...
21,39,2025-12-06-19-51-59 +0000,latimes,"Christmas Tree Lane will light up tonight, car...",https://www.latimes.com/california/story/2025-...
55,34,2025-12-06-16-33-22 +0000,nypost,Zohran Mamdani’s DSA party pushing China propa...,https://nypost.com/2025/12/06/us-news/dsa-push...
80,33,2025-12-06-14-35-09 +0000,nypost,Stephen Miller calls billion-dollar Minnesota ...,https://nypost.com/2025/12/06/us-news/stephen-...
149,31,2025-12-06-01-02-00 +0000,wsj,Two survivors of a Sept. 2 lethal U.S. strike ...,https://www.wsj.com/politics/national-security...
156,30,2025-12-06-00-22-01 +0000,nyt,San Diego Agrees to Pay $30 Million to Family ...,https://www.nytimes.com/2025/12/05/us/konoa-wi...
134,28,2025-12-06-02-38-00 +0000,wsj,How a Netflix-Warner Deal Would Change Everyth...,https://www.wsj.com/business/media/netflix-war...
40,27,2025-12-06-18-10-02 +0000,nypost,Deadly Hong Kong fire came after residents wer...,https://nypost.com/2025/12/06/world-news/hong-...
81,25,2025-12-06-14-31-08 +0000,cbc,Russia unleashes massive drone and missile att...,https://www.cbc.ca/news/world/russia-ukraine-a...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
